In [10]:
from tensorflow.keras.datasets   import boston_housing , mnist , fashion_mnist , reuters
from tensorflow.keras.models     import Sequential , clone_model , Model , load_model
from tensorflow.keras.layers     import Dense , Activation , InputLayer , Flatten , Input , BatchNormalization , Dropout , Embedding
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# CNN
from tensorflow.keras.layers     import Conv2D , MaxPooling2D , AveragePooling2D

# RNN
from tensorflow.keras.layers     import SimpleRNN , LSTM


from tensorflow.keras            import optimizers  
from tensorflow.keras.callbacks  import EarlyStopping , ModelCheckpoint , Callback
from tensorflow.keras.optimizers import SGD , Adam , RMSprop

# 이미지 로드
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

# 자연어 처리
from tensorflow.keras.preprocessing.text          import Tokenizer
from tensorflow.keras.preprocessing.sequence      import pad_sequences

from keras.utils.np_utils      import to_categorical

from sklearn.datasets          import load_iris , load_breast_cancer , load_digits
from sklearn.model_selection   import train_test_split
from sklearn.metrics           import accuracy_score
import matplotlib.pyplot as plt

import numpy  as np
import pandas as pd


In [30]:
import os
base_dir = '/tmp/beers/'
train_dir = os.path.join(base_dir,'train')

train_ganghan = os.path.join(train_dir,'ganghan')
train_gangseo = os.path.join(train_dir,'gangseo')
train_namsan = os.path.join(train_dir,'namsan')



In [31]:
train_ganghan_names = os.listdir(train_ganghan)
train_gangseo_names = os.listdir(train_gangseo)
train_namsan_names = os.listdir(train_namsan)


In [32]:
train_datagen = ImageDataGenerator(rescale = 1./255, # 모든 이미지 원소값들을 255로 나누기
                                   rotation_range=25, # 0~25도 사이에서 임의의 각도로 원본이미지를 회전
                                   width_shift_range=0.05, # 0.05범위 내에서 임의의 값만큼 임의의 방향으로 좌우 이동
                                   height_shift_range=0.05, # 0.05범위 내에서 임의의 값만큼 임의의 방향으로 상하 이동
                                   zoom_range=0.2, # (1-0.2)~(1+0.2) => 0.8~1.2 사이에서 임의의 수치만큼 확대/축소
                                   horizontal_flip=True, # 좌우로 뒤집기                                   
                                   vertical_flip=True,
                                   fill_mode='nearest'
                                  ) 

In [33]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size=16, # 한번에 변환된 이미지 16개씩 만들어라 라는 것
                                                    color_mode='grayscale', # 흑백 이미지 처리
                                                    class_mode='binary', 
                                                    target_size=(150,150))

Found 48 images belonging to 4 classes.
Found 18 images belonging to 4 classes.


In [34]:
train_generator.class_indices

{'.ipynb_checkpoints': 0, 'ganghan': 1, 'gangseo': 2, 'namsan': 3}

In [35]:
print('feature extraction - ')
beer_cnn_model = Sequential()

beer_cnn_model.add( Conv2D( filters = 32 , 
                            kernel_size = (3,3) , 
                            strides = (1,1) , 
                            padding = 'same' , activation = 'relu') ) 
beer_cnn_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'same' ) )
beer_cnn_model.add( Conv2D(filters = 64 , 
                            kernel_size = (3,3) , 
                            strides = (1,1) , 
                            padding = 'same' , activation = 'relu') ) 
beer_cnn_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'same' ) )
beer_cnn_model.add( Conv2D(filters = 64 , 
                            kernel_size = (3,3) , 
                            strides = (1,1) , 
                            padding = 'same' , activation = 'relu') ) 
beer_cnn_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'same' ) )



feature extraction - 


In [36]:
print('Dense 층 추가 - ')
beer_cnn_model.add( Flatten() )
beer_cnn_model.add( Dense(units = 512) )
beer_cnn_model.add( Activation('relu') )
beer_cnn_model.add( Dense(units = 256) )
beer_cnn_model.add( Activation('relu') )
beer_cnn_model.add( Dense(units = 128) )
beer_cnn_model.add( Activation('relu') )
beer_cnn_model.add( Dense(units = 64) )
beer_cnn_model.add( Activation('relu') )

beer_cnn_model.add( Dense(units = 10) )
beer_cnn_model.add( Activation('softmax') )


# mnist_cnn_model.output_shape


Dense 층 추가 - 


In [37]:
beer_cnn_model.compile(optimizer = Adam(learning_rate = 0.01), 
                      loss= 'sparse_categorical_crossentropy' , 
                      metrics=['accuracy'] ) 

In [41]:
beer_cnn_model_history = beer_cnn_model.fit(train_generator , epochs=50  , batch_size = 50 ,verbose = 1)

Epoch 1/50
3/3 [==============================] - 2s 631ms/step - loss: 0.8879 - accuracy: 0.5833
Epoch 2/50
3/3 [==============================] - 2s 608ms/step - loss: 0.8693 - accuracy: 0.6042
Epoch 3/50
3/3 [==============================] - 2s 595ms/step - loss: 1.0221 - accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 2s 651ms/step - loss: 0.9013 - accuracy: 0.5625
Epoch 5/50
3/3 [==============================] - 2s 612ms/step - loss: 0.9170 - accuracy: 0.5625
Epoch 6/50
3/3 [==============================] - 2s 603ms/step - loss: 0.9372 - accuracy: 0.5208
Epoch 7/50
3/3 [==============================] - 2s 595ms/step - loss: 0.9260 - accuracy: 0.5208
Epoch 8/50
3/3 [==============================] - 2s 607ms/step - loss: 0.9037 - accuracy: 0.5833
Epoch 9/50
3/3 [==============================] - 2s 599ms/step - loss: 0.8630 - accuracy: 0.5833
Epoch 10/50
3/3 [==============================] - 2s 615ms/step - loss: 0.7935 - accuracy: 0.6667
Epoch 11/50
3/3 [==

In [43]:
beer_cnn_model.evaluate(train_generator)

3/3 [==============================] - 1s 186ms/step - loss: 0.7028 - accuracy: 0.6458


[0.7028136253356934, 0.6458333134651184]